`Lab creado por Margarita Geleta para el curso Introducción a Machine Learning JEDI, edición 2021`

# [IX] Preprocesamiento de datos (Parte 2)

> Garbage in, Garbage out

### Valores nulos y NAs

Primero de todo, hemos de mirar si nuestro dataset está completo. Si tenemos valores nulos, debemos identificar como estan indentificados: `NA`, `?`, `-`, `-1`, `9999` ... En Python los valores missing se codifican como `NA`, si no los tratará como valores.

##### Ejemplo
En una encuesta se pregunta a las personas sobre su salario. Como más cobre, más probable es que responda con un `NA` y normalmente tenemos más personas que ganan menos. La realidad mostraría una media de `M`, aunque, como que tenemos muchos `NA`s debido al hecho que muchas personas que ganan mucho no conrtestan en la encuesta. Si eliminamos los `NA`s tendremos una media con cesgo `M' < M`. Así que eliminando los missings, obtenemos una media con cesgo.

#### Hay 3 tipos de `NA`s

- **MCAR** (Missing Completely At Random): los valores faltantes realmente son aleatorios. No sabemos la causa de su ausencia y no hay otras variables que la expliquen. No podemos estimar su valor. Si no son muchos valores faltantes, los podemos eliminar. Aunque a medida que vayamos eliminando más datos, perderemos precisión en nuestros modelos, por eso sería más interesante estimar los valores.

- **MAR** (Missing At Random): la probabilidad de la falta de un valor puede ser condicionada por otras variables. Podemos estimar el valor a partir de otras variables.

- **MNAR** (Missing Not At Random): la probabilidad de la falta de un valor depende directamente de otras variables.

#### Estrategias para afrontar `NA`s

- **Eliminar valores faltantes**: pero la precisión disminuye ya que el tamaño del dataset disminuye. Además la inferencia estadística puede tener cesgo si los `NA`s no son MCAR. 

- **Imputar los valores con un valor razonable**: se conoce como *single imputation*. Por ejemplo, imputar con la media de los valores de la columna. Pero puede ser mala idea si tenemos una clara relación entre las variables (por ej. caso lineal).

- **Imputar los valores a partir de un modelo**: podemos ajustar un modelo y imputar con valores predichos por el modelo (por ej. caso lineal, kNN). También podemos imputar con un cierto error, para que los puntos caigan no directamente en el modelo.

- **Imputar varias veces con subconjuntos de datos diferentes**: es un método más sofisticado, llamado *multiple imputation*. Imputar varias veces los valores faltantes y comparar que valores obtenemos.

#### ¿Cómo afrontar los ceros?

Los ceros es un tema delicado. Los ceros, a veces, pueden ser de hecho valores perdidos. Aparecen confusiones y complicaciones cuando algunos ceros son reales y otros son valores faltantes. Por ejemplo, en Excel, los datos faltantes se codifican con ceros por defecto. Hay que identificar si son datos nulos realmente o datos faltantes - la conclusión del estudio puede cambiar.


### Imputar con `scikit-sklearn`

`SimpleImputer`nos permite rellenar los valores perdidos con un simple valor, que podría ser la media, la mediana o el valor más frecuente, por ejemplo.

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')

df = pd.DataFrame({'A':[np.nan, 2, 3, np.nan, 1],
                   'B':[3, 5, np.nan, 6, 5.5],
                   'C':[0, np.nan, 3, np.nan, 1],
                   'D':[2, 6, 3, 4, 2],
                  })
df

,A,B,C,D
0,NaN,3.0,0.0,2
1,2.0,5.0,NaN,6
2,3.0,NaN,3.0,3
3,NaN,6.0,NaN,4
4,1.0,5.5,1.0,2


In [2]:
imp.fit_transform(df)

array([[2.        , 3.        , 0.        , 2.        ],
       [2.        , 5.        , 1.33333333, 6.        ],
       [3.        , 4.875     , 3.        , 3.        ],
       [2.        , 6.        , 1.33333333, 4.        ],
       [1.        , 5.5       , 1.        , 2.        ]])

In [3]:
imp.fit_transform(df['B'].to_numpy().reshape(-1, 1))

array([[3.   ],
       [5.   ],
       [4.875],
       [6.   ],
       [5.5  ]])

In [4]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp.fit_transform(df)

array([[2.  , 3.  , 0.  , 2.  ],
       [2.  , 5.  , 1.  , 6.  ],
       [3.  , 5.25, 3.  , 3.  ],
       [2.  , 6.  , 1.  , 4.  ],
       [1.  , 5.5 , 1.  , 2.  ]])

In [5]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imp.fit_transform(df)

array([[1. , 3. , 0. , 2. ],
       [2. , 5. , 0. , 6. ],
       [3. , 3. , 3. , 3. ],
       [1. , 6. , 0. , 4. ],
       [1. , 5.5, 1. , 2. ]])

In [6]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'constant', fill_value = 0)
imp.fit_transform(df)

array([[0. , 3. , 0. , 2. ],
       [2. , 5. , 0. , 6. ],
       [3. , 0. , 3. , 3. ],
       [0. , 6. , 0. , 4. ],
       [1. , 5.5, 1. , 2. ]])

`IterativeImputer` nos permite modelar los valores perdidos en función de otras variables, a través de un modelo.

In [7]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

- Primero de todo, nos imputa igual que antes con la opción `initial_strategy_parameter`. 
- Luego, entrena un modelo (definido en `default`).
- Y luego por cada columna, hace una predicción de sus valores perdidos partiendo de las demás columnas. El método se repite para cada combinación de columnas.

https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer

In [8]:
from sklearn.linear_model import LinearRegression
rimp = IterativeImputer(missing_values = np.nan, 
                        initial_strategy = 'mean', 
                        estimator = LinearRegression())
pd.DataFrame(rimp.fit_transform(df), columns = df.columns)

,A,B,C,D
0,0.339623,3.0,0.000000e+00,2.0
1,2.000000,5.0,-1.600000e+00,6.0
2,3.000000,12.0,3.000000e+00,3.0
3,1.698113,6.0,-1.086867e-08,4.0
4,1.000000,5.5,1.000000e+00,2.0


In [9]:
from sklearn.linear_model import ElasticNet
rimp = IterativeImputer(missing_values = np.nan, 
                        initial_strategy = 'mean', 
                        estimator = ElasticNet())
rimp.fit_transform(df)

array([[1.75359412, 3.        , 0.        , 2.        ],
       [2.        , 5.        , 1.4609086 , 6.        ],
       [3.        , 5.23234254, 3.        , 3.        ],
       [1.9856489 , 6.        , 1.71581949, 4.        ],
       [1.        , 5.5       , 1.        , 2.        ]])

Imputar con `KNNImputer`: dos observaciones son vecinas si los valores de sus variables que ambos no son `NA`s son similares.

In [10]:
from sklearn.impute import KNNImputer
kimp = KNNImputer(n_neighbors = 2)
kimp.fit_transform(df)

array([[2.  , 3.  , 0.  , 2.  ],
       [2.  , 5.  , 2.  , 6.  ],
       [3.  , 5.75, 3.  , 3.  ],
       [2.  , 6.  , 2.  , 4.  ],
       [1.  , 5.5 , 1.  , 2.  ]])

---

Vamos a ver el impacto que tiene la imputación a la hora de modelar datos.

In [11]:
df = pd.read_csv('data/Barcelona-Weather.csv')
df = df.drop(['STATION',
              'LATITUDE',
              'LONGITUDE',
              'ELEVATION',
              'SNWD',
              'PRCP_ATTRIBUTES',
              'SNWD_ATTRIBUTES',
              'TAVG_ATTRIBUTES',
              'TMAX_ATTRIBUTES',
              'TMIN_ATTRIBUTES'], axis = 1)
df.describe()

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,PRCP,TAVG,TMAX,TMIN
count,65970.000000,16360.000000,61187.000000,60985.000000
mean,1.612454,15.919719,19.346832,11.474661
std,6.539843,5.942094,6.476205,5.927301
min,0.000000,-2.100000,-1.800000,-10.000000
25%,0.000000,11.100000,14.400000,6.800000
50%,0.000000,15.300000,19.000000,11.000000
75%,0.000000,21.200000,24.800000,16.500000
max,186.700000,29.800000,38.400000,27.800000


In [12]:
df.head()

,NAME,DATE,PRCP,TAVG,TMAX,TMIN
0,"BARCELONA, SP",1920-01-01,3.5,NaN,NaN,NaN
1,"BARCELONA, SP",1920-01-02,0.0,NaN,NaN,NaN
2,"BARCELONA, SP",1920-01-03,0.0,NaN,NaN,NaN
3,"BARCELONA, SP",1920-01-04,16.3,NaN,NaN,NaN
4,"BARCELONA, SP",1920-01-05,7.9,NaN,NaN,NaN


- Selecciona solo aquellas filas que datan del año 1947 y son del Aeropuerto de Barelona.
- Piensa como podrías imputar los valores perdidos.
- Intenta modelar la precipitación en función de la temperatura, el mes y el día.


---

In [36]:
dff = df[(df['DATE'].str.contains('1947-..-..')) & (df['NAME'].str.contains('AEROPUERTO'))].drop(['NAME'], axis = 1)

In [37]:
print(df.shape, dff.shape)
dff.head()

(67809, 6) (365, 5)


,DATE,PRCP,TAVG,TMAX,TMIN
41943,1947-01-01,0.0,NaN,15.2,3.5
41944,1947-01-02,0.0,NaN,15.2,0.0
41945,1947-01-03,0.0,NaN,13.6,0.7
41946,1947-01-04,0.0,NaN,9.2,0.2
41947,1947-01-05,0.0,NaN,13.8,5.0


In [38]:
#Realizo la suma de las variables y verifico cuales son Nan
dff.isna().sum()

DATE      0
PRCP      0
TAVG    365
TMAX     10
TMIN     11
dtype: int64

In [39]:
#Sacamos el listado de los elementos Nan minimos
dff[dff.TMIN.isna()]

,DATE,PRCP,TAVG,TMAX,TMIN
42157,1947-08-03,0.0,NaN,NaN,NaN
42223,1947-10-08,0.0,NaN,NaN,NaN
42224,1947-10-09,12.2,NaN,NaN,NaN
42225,1947-10-10,0.0,NaN,NaN,NaN
42226,1947-10-11,0.0,NaN,NaN,NaN
42227,1947-10-12,3.6,NaN,NaN,NaN
42228,1947-10-13,0.0,NaN,NaN,NaN
42229,1947-10-14,0.0,NaN,NaN,NaN
42230,1947-10-15,0.0,NaN,NaN,NaN
42231,1947-10-16,0.0,NaN,NaN,NaN


In [40]:
Fechas = dff['DATE'].str.split('-',2, expand = True).rename(columns = {0:'YEAR', 1:'MONTH',2:'DAY'}).apply(pd.to_numeric).drop('YEAR', axis = 1)

In [41]:
Fechas

,MONTH,DAY
41943,1,1
41944,1,2
41945,1,3
41946,1,4
41947,1,5
...,...,...
42303,12,27
42304,12,28
42305,12,29
42306,12,30


In [42]:
dff = pd.merge(dff.drop('DATE', axis = 1), dates, left_index = True,right_index = True)

In [31]:
dff

,PRCP,TAVG,TMAX,TMIN,MONTH,DAY
41943,0.0,NaN,15.2,3.5,1,1
41944,0.0,NaN,15.2,0.0,1,2
41945,0.0,NaN,13.6,0.7,1,3
41946,0.0,NaN,9.2,0.2,1,4
41947,0.0,NaN,13.8,5.0,1,5
...,...,...,...,...,...,...
42303,0.0,NaN,11.5,2.0,12,27
42304,0.0,NaN,16.5,3.0,12,28
42305,0.0,NaN,12.0,6.0,12,29
42306,15.6,NaN,15.0,3.0,12,30


In [45]:
df1 = True
df1 = dff[pd.notnull(df['TMIN'])]
df1

<ipython-input-45-8bd181632930>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = dff[pd.notnull(df['TMIN'])]


,PRCP,TAVG,TMAX,TMIN,MONTH,DAY
41943,0.0,NaN,15.2,3.5,1,1
41944,0.0,NaN,15.2,0.0,1,2
41945,0.0,NaN,13.6,0.7,1,3
41946,0.0,NaN,9.2,0.2,1,4
41947,0.0,NaN,13.8,5.0,1,5
...,...,...,...,...,...,...
42303,0.0,NaN,11.5,2.0,12,27
42304,0.0,NaN,16.5,3.0,12,28
42305,0.0,NaN,12.0,6.0,12,29
42306,15.6,NaN,15.0,3.0,12,30


In [46]:
print(df1.shape)
#Hacemos una llamada de nuestra tabla con los datos de las columnas PRCP y TAVG
X = df1.drop([ 'PRCP', 'TAVG' ], axis = 1)
Y = df1['PRCP']

(354, 6)


In [49]:
from sklearn.metrics import mean_squared_error
modelo_lineal = LinearRegression()

In [52]:
#The cross_val_score () function will be used to perform the 
#evaluation, taking the dataset and cross-validation configuration
#and Returning a list of scores calculated for each fold.

from sklearn.model_selection import cross_val_score
Puntaje = -cross_val_score(modelo_lineal, X, Y, cv = 5, scoring = 'neg_mean_squared_error')
print(Puntaje.mean(), Puntaje.std())

12.735733030640992 15.73981908779669


# Buscamos valores perdidos


In [ ]:
imp = SimpleImputer(missing_values = np.man, strategy = 'mean')
dffi = imp.fit_transform(dff.drop(['TAVG','PRCP'], axis = 1))
X = dffi
Y = dff['PRCP']
modelo_lineal = LinearRegression()
scores = -cross_val_score()